<a href="https://colab.research.google.com/github/erayyildiz/ACL-Trends/blob/master/Electra_TR_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment


In [0]:
!pip install gdown &> /dev/null

!rm -rf * && rm -rf .* &> /dev/null
!unzip tr_models.zip -d . &> /dev/null
!rm tr_models.zip &> /dev/null
!mkdir datasets  &> /dev/null
!unzip MD.zip  -d datasets/ &> /dev/null
!rm MD.zip &> /dev/null

In [0]:
MODEL_NAME = 'bpe-stem-suffix-model'
DATA_DIR = '/content/electra_tr_model/'
TASKS_DATA_DIR = '/content/datasets/'

# Morphlogical Disambigation

In [0]:
from model import tokenization
from util import utils

tokenizer = tokenization.FullTokenizer(
        vocab_file=DATA_DIR + 'vocab.txt',
        do_lower_case=True)

In [0]:
from tqdm import tqdm_notebook as tqdm

train_file_path = TASKS_DATA_DIR + 'MD/data.train.txt'
num_lines = sum(1 for _ in open(train_file_path, 'r'))

sentences = []

with open(train_file_path, encoding='UTF8') as f:
    for line in tqdm(f, desc='Reading Data', total=num_lines):
        trimmed_line = line.strip(" \r\n\t")
        if trimmed_line.startswith("<S>") or trimmed_line.startswith("<s>"):
            sentence_words = []
        elif trimmed_line.startswith("</S>") or trimmed_line.startswith("</s>"):
                surface_words = []
                analyzes = []
                for word in sentence_words:
                    parses = word.split()
                    surface_words.append(parses[0])
                    analyzes.append(parses[1:])

                sentences.append((surface_words, analyzes))
                break
        elif len(trimmed_line) == 0 or "<DOC>" in trimmed_line or trimmed_line.startswith(
                "</DOC>") or trimmed_line.startswith(
                "<TITLE>") or trimmed_line.startswith("</TITLE>"):
            pass
        else:
            sentence_words.append(trimmed_line)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [0]:
for words, analyzes in sentences:
    for word, cur_analyses in zip(words, analyzes):
        word = word.replace('İ', 'i').replace('I', 'ı').lower()
        print(tokenizer.wordpiece_tokenizer.tokenize(word))
        print(cur_analyses)
        print()

['refah']
['Refah+Noun+Prop+A3sg+Pnon+Nom', 'refah+Noun+A3sg+Pnon+Nom']

['da']
['da+Conj']

['türkçe']
['Türkçe+Noun+Prop+A3sg+Pnon+Nom', 'türkçe+Adj', 'türk+Noun+A3sg+Pnon+Equ', 'türk+Adj^DB+Adverb+Ly', 'türk+Adj^DB+Adj+AsIf']

['için']
['için+Postp+PCNom', 'iç+Noun+A3sg+P2sg+Nom', 'iç+Noun+A3sg+Pnon+Gen', 'iç+Verb+Pos+Imp+A2pl']

['görüş']
['görüş+Noun+A3sg+Pnon+Nom', 'gör+Verb+Recip+Pos+Imp+A2sg', 'gör+Verb+Pos^DB+Noun+Inf3+A3sg+Pnon+Nom', 'görüş+Verb+Pos+Imp+A2sg']

['iste', '##di']
['iste+Verb+Pos+Past+A3sg']



In [0]:
tokenizer.tokenize('çok öğe Açık Her şey ve herkes yarım kaldı. Bu süreç , kültür sanat alanında bazı kurumları dijitale kaydırırken bazılarıysa dijitale mesafeli durmayı tercih etti.')

['çok',
 'öğe',
 'açık',
 'her',
 'şey',
 've',
 'herkes',
 'yarım',
 'kaldı',
 '.',
 'bu',
 'süreç',
 ',',
 'kültür',
 'sanat',
 'alan',
 '##ında',
 'bazı',
 'kurum',
 '##ları',
 'dijital',
 '##e',
 'kaydı',
 '##r',
 '##ırken',
 'bazı',
 '##larıysa',
 'dijital',
 '##e',
 'mesafe',
 '##li',
 'durma',
 '##yı',
 'tercih',
 'et',
 '##ti',
 '.']

In [0]:
from model import modeling
from util import training_utils
import configure_pretraining

config = configure_pretraining.PretrainingConfig(MODEL_NAME, DATA_DIR)
model_fn = model_fn_builder(config=config)


bert_config = training_utils.get_bert_config(config)

bert_model = modeling.BertModel(
        bert_config=bert_config,
        is_training=True,
        input_ids=features["input_ids"],
        input_mask=features["input_mask"],
        token_type_ids=features["segment_ids"],
        use_one_hot_embeddings=config.use_tpu,
        embedding_size=config.embedding_size)

NameError: ignored